Santa Gift Matching Challenge
-------------------------------------------

1,000,000명의 아이가 있고, 각각 아이들이 받고 싶어하는 100가지 종류의 선물 리스트가 있다.
내가 가지고 있는 선물은 총 1000종류이고 1000명의 '착한' 아이가 있다.

내 목표는 1,000,000명의 모든 아이들에게 알맞는 선물을 주어서 모두가 만족하게 하는 것이다. 즉, 선물은 주는 나(산타)와 아이들 모두가 만족해야 한다. 아이들은 그들의 wish list에 있는 선물들 중에서 가장 높은 순위를 가진 선물을 가질 수록 행복하고, 산타는 아이가 '착한 아이' 리스트에 높은 순위에 있을 수록 행복하다.

#### 몇 가지 주의사항들이 있다.
1. 전체 아이들의 5%는 세쌍둥이이다. 0,1,2 ~ 4998, 4999, 5000는 세쌍둥이이다. 세쌍둥이는 그들이 서로 다른 선호도를 지녔을지라도 같은 선물을 주어야 한다.
2. 전체 아이들의 4%는 쌍둥이이다. 5000, 5001 ~ 44999, 45000는 쌍둥이이다. 쌍둥이는 그들이 서로 다른 선호도를 지녔을지라도 같은 선물을 주어야 한다.
3. 각 선물은 1000개를 줄 수 있다. 즉 1000(선물의 종류)*1000(각 선물의 양)=1,000,000(선물을 주어야 될 아이의 수, 줄 수 있는 선물은 1000개를 넘길 수 없다.
4. Average Normalized Happiness에 결과값을 저장한다.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import operator
import math

In [2]:
INPUT_PATH = '../input/'


def lcm(a, b):
    # math.gcd(a,b) = a와 b의 최대공약수
    return a * b // math.gcd(a, b)

In [3]:
#gift = gift_goodkids, wisht = child_wishtist, pred = 
def avg_normalized_happiness(pred, gift, wish):
    
    n_children = 1000000 # n children to give
    n_gift_type = 1000 # n types of gifts available
    n_gift_quantity = 1000 # each type of gifts are limited to this quantity
    n_gift_pref = 100 # number of gifts a child ranks
    n_child_pref = 1000 # number of children a gift ranks
    twins = math.ceil(0.04 * n_children / 2.) * 2    # 전체 인구의 4%가 쌍둥이
    triplets = math.ceil(0.005 * n_children / 3.) * 3    # 전체 인구의 0.5%가 세쌍둥이
    ratio_gift_happiness = 2 #리스트에 있는 아이에게 선물을 주었을 때 받는 happiness
    ratio_child_happiness = 2 #리스트에 있는 선물을 받았을 때 받는 happiness

    # 1. 세쌍둥이가 같은 선물을 가졌는지 확인
    for t1 in np.arange(0, triplets, 3): #0부터 triplets(세 쌍둥이의 수)까지 3씩 증가하는 sequence 생성
        triplet1 = pred[t1]
        triplet2 = pred[t1+1]
        triplet3 = pred[t1+2]
        # print(t1, triplet1, triplet2, triplet3)
        assert triplet1 == triplet2 and triplet2 == triplet3 #assert condition : condition에 맞지 않으면 error을 반환
                
    # 2. 쌍둥이가 같은 선물을 가졌는지 확인
    for t1 in np.arange(triplets, triplets+twins, 2): #0~5000까지가 세쌍둥이, 5000~45000이 쌍둥이, 위와 같은 방식으로 sequence 생성
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        assert twin1 == twin2

    max_child_happiness = n_gift_pref * ratio_child_happiness #아이가 받을 수 있는 최대 행복 = 아이가 선호하는 선물의 수 * child happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness #선물이 줄 수 있는 최대 행복 = 선물이 선호하는 아이의 수 * gift happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for i in range(len(pred)):
        child_id = i
        gift_id = pred[i]
        
        # child_id와 gifr_id가 존재하는지 확인
        assert child_id < n_children #아이의 수보다 child_id가 작아야 한다
        assert gift_id < n_gift_type #줄 수 있는 선물의 수보다 gift_id가 작아야 한다
        assert child_id >= 0  #각각의 id는 0부터 시작이 된다.
        assert gift_id >= 0
    
        #np.where(wish[child_id]==gift_id)[0] = where 조건절을 만족하는 아이가 가장 받고 싶어하는 선물
        child_happiness = (n_gift_pref - np.where(wish[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        #np.where(gift[gift_id]==child_id[0]) = where 조건절을 만족하는 각 선물을 가장 주고 싶어하는 아이
        gift_happiness = ( n_child_pref - np.where(gift[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
        
    denominator1 = n_children*max_child_happiness #모든 아이의 최대 행복치
    denominator2 = n_gift_quantity*max_gift_happiness*n_gift_type #모든 선물의 최대 행복치
    common_denom = lcm(denominator1, denominator2) #denominator1, denominator2 최소 공배수
    multiplier = common_denom / denominator1 

    #math.pow(  ) : 거듭제곱함수
    ret = float(math.pow(total_child_happiness*multiplier,3) + \
        math.pow(np.sum(total_gift_happiness),3)) / float(math.pow(common_denom,3))
    return ret

In [4]:
def get_overall_hapiness(wish, gift):


    res_child = dict()
    for i in range(0, wish.shape[0]):
        for j in range(55):
            res_child[(i, wish[i][j])] = int(100* (1 + (wish.shape[1] - j)*2))

    res_santa = dict()
    for i in range(gift.shape[0]):
        for j in range(gift.shape[1]):
            res_santa[(gift[i][j], i)] = int((1 + (gift.shape[1] - j)*2))

    #set( ) : 자료의 집합형.
    #res_satna와 res_child의 key의 자료 집합형을 모아 postivie_cases 변수에 저장
    positive_cases = list(set(res_santa.keys()) | set(res_child.keys()))
    print('Positive case tuples (child, gift): {}'.format(len(positive_cases)))

    res = dict()
    for p in positive_cases:
        res[p] = 0
        if p in res_child:
            res[p] += res_child[p]
        if p in res_santa:
            res[p] += res_santa[p]
    return res

In [5]:
#list에서 키가 아닌 값으로 reversed_sorting
def sort_dict_by_values(a, reverse=True):
    sorted_x = sorted(a.items(), key=operator.itemgetter(1), reverse=reverse)
    return sorted_x


def value_counts_for_list(lst):
    a = dict(Counter(lst))
    a = sort_dict_by_values(a, True)
    return a


#np.ravel( ) : 다차원인 array를 1차원 배열로 평평하게 펼쳐줌
def get_most_desired_gifts(wish, gift):
    best_gifts = value_counts_for_list(np.ravel(wish))
    return best_gifts


def recalc_hapiness(happiness, best_gifts, gift):
    recalc = dict()
    for b in best_gifts:
        recalc[b[0]] = b[1] / 2000000

    for h in happiness:
        c, g = h
        happiness[h] /= recalc[g]
        
    return happiness

In [6]:
def solve():
    wish = pd.read_csv(INPUT_PATH + 'child_wishlist_v2.csv', header=None).as_matrix()[:, 1:]#프레임을 numpy배열 표현으로 변환 반환 값 : ndarray
    gift_init = pd.read_csv(INPUT_PATH + 'gift_goodkids_v2.csv', header=None).as_matrix()[:, 1:]#numpy 배열로 변환하지만 인덱스는 유지
    gift = gift_init.copy() #원본 리스트 요소에 영향을 끼치지 않기 위해서
    answ = np.zeros(len(wish), dtype=np.int32)
    answ[:] = -1
    gift_count = np.zeros(len(gift), dtype=np.int32)

    happiness = get_overall_hapiness(wish, gift)
    best_gifts = get_most_desired_gifts(wish, gift)
    happiness = recalc_hapiness(happiness, best_gifts, gift)
    sorted_hapiness = sort_dict_by_values(happiness)
    print('Happiness sorted...')

    for i in range(len(sorted_hapiness)):
        child = sorted_hapiness[i][0][0]
        g = sorted_hapiness[i][0][1]
        if answ[child] != -1:
            continue
        if gift_count[g] >= 1000:
            continue
        if child <= 5000 and gift_count[g] < 997: # 세쌍둥이의 경우
            if child % 3 == 0: #3으로 나눠떨어지면 다 같은 선물
                answ[child] = g
                answ[child+1] = g
                answ[child+2] = g
                gift_count[g] += 3 #gift_count가 3 증가
            elif child % 3 == 1: #나머지가 1이면 전사람과 다음사람이랑 같은 선물
                answ[child] = g
                answ[child-1] = g
                answ[child+1] = g
                gift_count[g] += 3
            else:
                answ[child] = g
                answ[child-1] = g
                answ[child-2] = g
                gift_count[g] += 3
        elif child > 5000 and child <= 45000 and gift_count[g] < 998: #쌍둥이 경우
            if child % 2 == 0:
                answ[child] = g
                answ[child - 1] = g
                gift_count[g] += 2
            else:
                answ[child] = g
                answ[child + 1] = g
                gift_count[g] += 2
        elif child > 45000:
            answ[child] = g
            gift_count[g] += 1

    print('Left unhappy:', len(answ[answ == -1]))
    
    # unhappy children
    for child in range(45001, len(answ)):
        if answ[child] == -1:
            g = np.argmin(gift_count) # np.argmin
            answ[child] = g
            gift_count[g] += 1

    if answ.min() == -1:
        print('Some children without present')
        exit()

    if gift_count.max() > 1000:
        print('Some error in kernel: {}'.format(gift_count.max()))
        exit()

    print('Start score calculation...')
    # score = avg_normalized_happiness(answ, gift_init, wish)
    # print('Predicted score: {:.8f}'.format(score))
    score = avg_normalized_happiness(answ, gift, wish)
    print('Predicted score: {:.8f}'.format(score))

    out = open('subm_{}.csv'.format(score), 'w') # 
    out.write('ChildId,GiftId\n')
    for i in range(len(answ)):
        out.write(str(i) + ',' + str(answ[i]) + '\n')
    out.close()


if __name__ == '__main__':
    solve()

FileNotFoundError: File b'../input/child_wishlist_v2.csv' does not exist

In [ ]:
answ

### pandas.DataFrame.as_matrix()

* 데이터프레임을 numpy 배열 표현으로 변환하여 반환
* numpy 배열로 변환하지만 인덱스는 그대로 유지

In [ ]:
#pandas.DataFrame.as_matrix() 예시
index = [1, 2, 3, 4, 5, 6, 7]
a1 = [np.nan, np.nan, np.nan, 0.1, 0.1, 0.1, 0.1]
b1 = [0.2, np.nan, 0.2, 0.2, 0.2, np.nan, np.nan]
c1 = [np.nan, 0.5, 0.5, np.nan, 0.5, 0.5, np.nan]
df = pd.DataFrame({'A': a1, 'B': b1, 'C': c1}, index=index)
df = df.rename_axis('ID')

In [ ]:
df

In [ ]:
numpy_matrix=df.as_matrix()
numpy_matrix

### numpy.argmin()

axis에 따른 최소 인덱스를 반환
* (a, axis=None, out=None)
* 최소값이 여러 번 나오는 경우 첫 번째 발생에 해당하는 인덱스 반환

In [ ]:
a = np . arange ( 6 ) . reshape ( 2 , 3 )
a

In [ ]:
np.argmin(a)

In [ ]:
np.argmin(a,axis = 0)

In [ ]:
np.argmin(a,axis = 1)

### 파일 생성하기

f = open("새파일.txt", 'w')

f.close()

파일을 생성하기 위해 open이라는 파이썬 내장 함수 사용. open 함수는 다음과 같이 "파일 이름"과 "파일 열기 모드"를 입력값으로 받고 결과값으로 파일 객체를 반환

파일 객체 = open(파일 이름, 파일 열기 모드)

파일열기모드
* r	읽기모드 - 파일을 읽기만 할 때 사용
* w	쓰기모드 - 파일에 내용을 쓸 때 사용
* a	추가모드 - 파일의 마지막에 새로운 내용을 추가 시킬 때 사용

In [ ]:
#파일을 쓰기 모드로 열게 되면 해당 파일이 이미 존재할 경우 원래 있던 내용이 모두 사라지고, 해당 파일이 존재하지 않으면 새로운 파일이 생성된다.